In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
# RDD con información de vuelos programados con la forma (número de vuelo,
# código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de
# salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada
# HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron
# ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de
# salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado).
# Con base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá
# información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información
# de la llegada

In [4]:
rdd_vuelosProgramados = [
    (345,'AAR','MDQ','EZE','20191003','10:00','20191003','11:00'),
    (355,'AAR','MDQ','AEP','20191104','10:15','20191104','11:00'),
    (315,'APY','EPA','MDQ','20191104','10:15','20191104','11:00'),
    (225,'FLG','EPA','MDZ','20191104','10:15','20191104','11:00'),
    (255,'AAR','MDQ','MDZ','20191105','10:15','20191105','11:00'),
    (175,'AAR','MDQ','AEP','20191103','10:15','20191103','11:00'),
    (255,'AAR','MDQ','AEP','20191104','10:15','20191104','11:00'),
    (155,'AAR','MDQ','AEP','20191105','10:15','20191105','11:00'),
    (455,'AAR','MDQ','AEP','20191106','10:15','20191106','11:00'),
    (535,'AAR','MDQ','AEP','20191107','10:15','20191107','11:00')

]


In [5]:
data = sc.parallelize(rdd_vuelosProgramados);

In [6]:
#Rta A el aeropuerto con mayor transito
data1=data.map(lambda x: (x[2],1))
data1.collect()

[('MDQ', 1),
 ('MDQ', 1),
 ('EPA', 1),
 ('EPA', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1)]

In [7]:
data2=data.map(lambda x: (x[3],1))
data2.collect()

[('EZE', 1),
 ('AEP', 1),
 ('MDQ', 1),
 ('MDZ', 1),
 ('MDZ', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1)]

In [8]:
data3=data1+data2
data3.collect()

[('MDQ', 1),
 ('MDQ', 1),
 ('EPA', 1),
 ('EPA', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('MDQ', 1),
 ('EZE', 1),
 ('AEP', 1),
 ('MDQ', 1),
 ('MDZ', 1),
 ('MDZ', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1),
 ('AEP', 1)]

In [9]:
data4=data3.reduceByKey(lambda x , y : (x+y))
data5=data4.collect()
data5

[('AEP', 6), ('MDZ', 2), ('MDQ', 9), ('EPA', 2), ('EZE', 1)]

In [10]:
def getValue(item):   
    return item[1]
resultadoFinal=sorted(data5, key=getValue)
resultadoFinal.reverse()
resultadoFinal

[('MDQ', 9), ('AEP', 6), ('EPA', 2), ('MDZ', 2), ('EZE', 1)]

In [14]:
resultadoFinal[0]

('MDQ', 9)

In [11]:
#Esto no funciona ordena por clave 
data4.reduce(lambda x ,y : x if x > y else y )

('MDZ', 2)

In [19]:
#O sino puedo hacer lo siguiente para devolver los resultados ordenados
resultado=data3.reduceByKey(lambda x , y : (x+y)).map(lambda x: (x[1],x[0])).reduce(lambda x ,y : x if x > y else y )
resultadoRta=(resultado[1],resultado[0])
resultadoRta

('MDQ', 9)

In [36]:
#Mas simmple sino :
resultado2=data3.reduceByKey(lambda x , y : (x+y)).reduce(lambda x, y: x if x[1] > y[1] else y)
resultado2

('MDQ', 9)

In [29]:
#Rta B la aerolinea con mayor cantidad de vuelos
dataB=data.map(lambda x: (x[1],1)).reduceByKey(lambda x , y : (x+y)).map(lambda x: (x[1],x[0])).reduce(lambda x ,y : x if x > y else y )
resultadoRtaB=(dataB[1],dataB[0])
resultadoRtaB

('AAR', 8)

In [37]:
#Mas simmple sino :
resultadoRtaB=data.map(lambda x: (x[1],1)).reduceByKey(lambda x , y : (x+y)).reduce(lambda x, y: x if x[1] > y[1] else y)
resultadoRtaB

('AAR', 8)